In [15]:
import glob
import json

from os import path

import numpy as np
import pandas as pd
import scipy.stats

In [18]:
exp_path = "experiments/baselines/*/*"
exps = glob.glob(exp_path)
print("Number of experiments found: {}".format(len(exps)))
finished_exps = [
    exp for exp in exps if len(glob.glob(path.join(exp, "*"))) == 5]
not_finished_exps = [
    exp for exp in exps if len(glob.glob(path.join(exp, "*"))) != 5]
print("Number of experiments finished: {}".format(len(finished_exps)))
print("Unfinished experiments: {}".format(not_finished_exps))
hyperparam_names = sorted([
    "num_classes",
    "num_epochs",
    "use_last_n_layers",
    "no_use_pretrained",
    "lr",
    "model_name", 
    "dataset_name",
    "batch_size"
])
results = {}
agg_results = {}
metrics = [
    # "val_acc", "val_weighted_f1", 
    "test_acc", "test_weighted_f1"]
for exp in finished_exps:
    stat_fpath = path.join(exp, "test_stats.json")
    if not path.isfile(stat_fpath):
        print("{} does not exist.".format(stat_fpath))
        continue
    else:
        with open(stat_fpath) as f:
            stats = json.load(f)
        with open(path.join(exp, "opts.json")) as f:
            opts = json.load(f)
            
        model_opts = tuple([opts[k] for k in hyperparam_names])
        if model_opts not in results:
            results[model_opts] = {k: [] for k in metrics}
        for k in metrics:
            if "f1" in k:
                results[model_opts][k].append(stats[k] * 100)
            else:
                results[model_opts][k].append(stats[k])

for k, stats in results.items():
    if k not in agg_results:
        agg_results[k] = {}
    for name, vals in stats.items():
        agg_results[k][name + "_mean"] = np.round(np.mean(vals), 2)
        agg_results[k][name + "_std"] = np.round(scipy.stats.sem(vals), 2)

header = list(hyperparam_names) + [
    k + pf for k in metrics for pf in ["_mean", "_std"]]

df_rows = []
for k, stats in agg_results.items():
    df_rows.append(list(k))
    for name in metrics:
        df_rows[-1].append(stats[name + "_mean"])
        df_rows[-1].append(stats[name + "_std"])
df = pd.DataFrame(df_rows, columns=header)

Number of experiments found: 540
Number of experiments finished: 540
Unfinished experiments: []


In [24]:
df[
    # (df.model_name == "wide_resnet50_2") &
    (df.no_use_pretrained == False) &
    (df.use_last_n_layers == -1)
].sort_values(by=["dataset_name", "model_name"])

,batch_size,dataset_name,lr,model_name,no_use_pretrained,num_classes,num_epochs,use_last_n_layers,test_acc_mean,test_acc_std,test_weighted_f1_mean,test_weighted_f1_std
56,64,v1_b2p_rgb_large_1150_600_jpg,0.001,efficientnet_v2_m,False,2,200,-1,61.02,0.86,60.19,1.26
23,64,v1_b2p_rgb_large_1150_600_jpg,0.001,efficientnet_v2_s,False,2,200,-1,61.32,1.23,60.61,1.68
94,64,v1_b2p_rgb_large_1150_600_jpg,0.001,resnet18,False,2,200,-1,63.33,0.89,62.97,1.00
149,64,v1_b2p_rgb_large_1150_600_jpg,0.001,resnet50,False,2,200,-1,65.16,0.28,65.01,0.29
123,64,v1_b2p_rgb_large_1150_600_jpg,0.001,wide_resnet50_2,False,2,200,-1,60.22,1.46,60.12,1.43
60,64,v1_b2p_rgb_large_2350_1200_jpg,0.001,efficientnet_v2_m,False,2,200,-1,54.95,1.65,54.61,1.79
6,64,v1_b2p_rgb_large_2350_1200_jpg,0.001,efficientnet_v2_s,False,2,200,-1,59.30,1.68,58.67,1.48
77,64,v1_b2p_rgb_large_2350_1200_jpg,0.001,resnet18,False,2,200,-1,58.25,1.61,57.91,1.56
173,64,v1_b2p_rgb_large_2350_1200_jpg,0.001,resnet50,False,2,200,-1,57.39,1.49,57.06,1.51
112,64,v1_b2p_rgb_large_2350_1200_jpg,0.001,wide_resnet50_2,False,2,200,-1,52.53,0.94,52.15,1.29


In [12]:
df[df.model_name == "wide_resnet50_2"].groupby([
    "dataset_name", 
    "lr", 
    "no_use_pretrained", 
    "num_classes",
    "num_epochs",
    "batch_size",
    "use_last_n_layers",
    "model_name"
]).max("val_acc_mean")

val_acc_mean  \
dataset_name                   lr    no_use_pretrained num_classes num_epochs batch_size use_last_n_layers model_name                      
v1_b2p_rgb_large_1150_600_jpg  0.001 False             2           200        64         1                 wide_resnet50_2         59.02   
                                                                                         9                 wide_resnet50_2         59.44   
v1_b2p_rgb_large_2350_1200_jpg 0.001 False             2           200        64         1                 wide_resnet50_2         56.13   
                                                                                         9                 wide_resnet50_2         61.02   
v1_b2p_rgb_large_590_320_jpg   0.001 False             2           200        64         1                 wide_resnet50_2         59.30   
                                                                                         9                 wide_resnet50_2         59.02   
v2_b2p_rgb_large_1150_600_jpg  0.001 False             2           200        64         1                 wide_resnet50_2         58.88   
                                                                                         9                 wide_resnet50_2         61.11   
v2_b2p_rgb_large_2350_1200_jpg 0.001 False             2           200        64         1                 wide_resnet50_2         61.56   
                                                                                         9                 wide_resnet50_2         62.64   
v2_b2p_rgb_large_590_320_jpg   0.001 False             2           200        64         1                 wide_resnet50_2         58.81   
                                                                                         9                 wide_resnet50_2         59.71   

                                                                                                                            val_acc_std  \
dataset_name                   lr    no_use_pretrained num_classes num_epochs batch_size use_last_n_layers model_name                     
v1_b2p_rgb_large_1150_600_jpg  0.001 False             2           200        64         1                 wide_resnet50_2         0.42   
                                                                                         9                 wide_resnet50_2         0.70   
v1_b2p_rgb_large_2350_1200_jpg 0.001 False             2           200        64         1                 wide_resnet50_2         0.68   
                                                                                         9                 wide_resnet50_2         0.93   
v1_b2p_rgb_large_590_320_jpg   0.001 False             2           200        64         1                 wide_resnet50_2         0.61   
                                                                                         9                 wide_resnet50_2         0.35   
v2_b2p_rgb_large_1150_600_jpg  0.001 False             2           200        64         1                 wide_resnet50_2         0.74   
                                                                                         9                 wide_resnet50_2         0.68   
v2_b2p_rgb_large_2350_1200_jpg 0.001 False             2           200        64         1                 wide_resnet50_2         0.86   
                                                                                         9                 wide_resnet50_2         0.54   
v2_b2p_rgb_large_590_320_jpg   0.001 False             2           200        64         1                 wide_resnet50_2         0.41   
                                                                                         9                 wide_resnet50_2         0.33   

                                                                                                                            val_weighted_f1_mean  \
dataset_name                   lr    no_use_pretrained num_classes n